# Machine-Assisted Identity Proving

q-Kangaroo provides a complete algorithmic pipeline for **proving q-hypergeometric
identities** -- from discovering recurrences to extracting closed forms and
verifying proofs. The pipeline has three main stages:

1. **Indefinite summation** (`q_gosper`): Check if a q-hypergeometric term has an antidifference.
2. **Definite summation** (`q_zeilberger` + `verify_wz`): Find a linear recurrence for a definite sum, with a computer-verifiable WZ proof certificate.
3. **Closed-form recovery** (`q_petkovsek`): Solve the recurrence to express the sum as a ratio of q-Pochhammer products.

Two extensions complete the toolkit:
- `prove_nonterminating`: Prove nonterminating identities via the Chen-Hou-Mu (2010) specialization method.
- `find_transformation_chain`: Search for a chain of known transformations (Heine, Sears, Watson) connecting two equivalent series.

**Note on concrete $q$ values.** All algorithms in this notebook work with concrete
rational numbers $q \in \mathbb{Q}$ (e.g., $q = 2$), not a formal variable.
This is because the underlying algorithms require polynomial arithmetic over
$\mathbb{Q}[x]$ with $x = q^n$ substituted at concrete values. The results are
exact (all arithmetic is rational, not floating-point).

## q-Gosper: Indefinite Summation

The **q-Gosper algorithm** (Paule, 1995) determines whether a q-hypergeometric
term $t_k$ has a q-hypergeometric **antidifference** -- a term $z_k$ such that
$t_k = z_{k+1} - z_k$. If successful, the sum $\sum_{k=0}^{n} t_k = z_{n+1} - z_0$
can be evaluated in closed form.

This is the q-analog of Gosper's algorithm for ordinary hypergeometric summation.
The algorithm factors the term ratio $t_{k+1}/t_k$ into coprime polynomials and
solves a functional equation for the certificate.

In [ ]:
from q_kangaroo import q_gosper

# q-Vandermonde: 2phi1(q^{-3}, q^2; q^3; q, q^4) at q=2
# Parameters: upper = [(1,1,-3), (1,1,2)], lower = [(1,1,3)], z = q^4
result = q_gosper([(1,1,-3), (1,1,2)], [(1,1,3)], 1, 1, 4, 2, 1)
print("Summable:", result["summable"])
print("Certificate:", result["certificate"])

Summable: True
Certificate: (-64/465*x^2 + 26/155*x - 1/30) / (x - 1/2)


In [ ]:
# Non-summable: 2phi1(q^{-3}, q^3; q^7; q, q^2) at q=2
# This series has no q-hypergeometric antidifference.
result2 = q_gosper([(1,1,-3), (1,1,3)], [(1,1,7)], 1, 1, 2, 2, 1)
print("Summable:", result2["summable"])

Summable: False


When q-Gosper reports `summable: False`, the sum may still satisfy a **recurrence**.
This is where q-Zeilberger takes over -- it finds recurrences for definite sums
even when no closed-form antidifference exists.

## q-Zeilberger: Creative Telescoping

The **q-Zeilberger algorithm** (Koornwinder, 1993) finds a linear recurrence

$$c_0 S(n) + c_1 S(n+1) + \ldots + c_d S(n+d) = 0$$

for the definite sum $S(n) = \sum_k F(n,k)$ where $F$ is q-hypergeometric in both
$n$ and $k$. It uses q-Gosper as a subroutine (creative telescoping) and also
produces a **WZ proof certificate** $R(n,k)$.

In [ ]:
from q_kangaroo import q_zeilberger

# q-Vandermonde: 2phi1(q^{-5}, q^2; q^3; q, q^4) at n=5, q=2
# The n-dependent parameter q^{-5} is auto-detected.
result = q_zeilberger([(1,1,-5), (1,1,2)], [(1,1,3)], 1, 1, 4, 5, 2, 1, 3)
print("Found recurrence:", result["found"])
print("Recurrence order:", result["order"])
print("Coefficients:", result["coefficients"])

Found recurrence: True
Recurrence order: 1
Coefficients: [Fraction(-21, 85), Fraction(1, 1)]


A **first-order recurrence** $c_0 S(n) + c_1 S(n+1) = 0$ means
$S(n+1)/S(n) = -c_0/c_1$, giving the ratio of consecutive sums directly.
Here $S(n+1)/S(n) = -(-21/85)/1 = 21/85$.

In [ ]:
# Higher-complexity example: 3phi2(q^{-5}, q, q^2; q^3, q^4; q, q^3)
result2 = q_zeilberger(
    [(1,1,-5), (1,1,1), (1,1,2)], [(1,1,3), (1,1,4)],
    1, 1, 3, 5, 2, 1, 3
)
print("Found recurrence:", result2["found"])
print("Recurrence order:", result2["order"])
print("Coefficients:", result2["coefficients"])

Found recurrence: True
Recurrence order: 1
Coefficients: [Fraction(-1336468685307, 1666075405115), Fraction(1, 1)]


The `max_order` parameter (here 3) controls how high the algorithm searches
for a recurrence. If no recurrence is found within `max_order`, try increasing it.
Note the large rational coefficients for the ${}_3\phi_2$ -- this is typical
of exact rational arithmetic over $\mathbb{Q}$.

## WZ Certificates: Computer-Verified Proofs

The **WZ (Wilf-Zeilberger) certificate** $R(n,k)$ is a rational function that
provides a **computer-verifiable proof** of the summation identity. The key
identity is the telescoping equation:

$$F(n+1,k) - F(n,k) = G(n,k+1) - G(n,k)$$

where $G(n,k) = R(n,k) \cdot F(n,k)$. Summing both sides over $k$ gives the
recurrence, since the right side telescopes. The `verify_wz` function checks
this identity at multiple evaluation points.

In [ ]:
from q_kangaroo import verify_wz

# Verify the q-Vandermonde identity with 20 evaluation points
result = verify_wz(
    [(1,1,-5), (1,1,2)], [(1,1,3)],  # upper, lower
    1, 1, 4,                           # z = q^4
    5,                                 # n = 5
    2, 1,                              # q = 2
    3,                                 # max_order = 3
    20                                 # check 20 evaluation points
)
print("Verified:", result["verified"])
print("Recurrence order:", result["order"])
print("Certificate:", result["certificate"])

Verified: True
Recurrence order: 1
Certificate: (-127/9999360*x^5 + 677333/849945600*x^4 - 344789/4999680*x^3 + 154813/1249920*x^2 - 187037/2656080*x + 12233/830025) / (x - 1/2)


The verification is **exact** -- all arithmetic is over $\mathbb{Q}$, not
floating-point. The certificate $R(n,k)$ is a rational function in $x = q^k$
whose numerator and denominator are polynomials with rational coefficients.

This provides a rigorous computer-verified proof: even if we only had the
certificate from a published paper, we could independently verify it by checking
the telescoping identity at concrete evaluation points.

In [ ]:
# The certificate is a rational function in x = q^k
# We can inspect its numerator and denominator polynomials
zeil = q_zeilberger([(1,1,-5), (1,1,2)], [(1,1,3)], 1, 1, 4, 5, 2, 1, 3)
print("Certificate numerator:", zeil["numer"])
print("Certificate denominator:", zeil["denom"])

Certificate numerator: -127/9999360*x^5 + 677333/849945600*x^4 - 344789/4999680*x^3 + 154813/1249920*x^2 - 187037/2656080*x + 12233/830025
Certificate denominator: x - 1/2


## q-Petkovsek: Solving Recurrences for Closed Forms

Once q-Zeilberger finds a recurrence $c_0 S(n) + c_1 S(n+1) + \ldots = 0$,
the **q-Petkovsek algorithm** (Petkovsek, 1992; Abramov-Paule-Petkovsek, 1998)
solves it to recover a **closed-form solution** as a ratio of q-Pochhammer products:

$$y(n) = s \cdot q^{c\binom{n}{2}} \cdot \frac{\prod_i (q^{a_i}; q)_n}{\prod_j (q^{b_j}; q)_n}$$

This completes the pipeline: **series $\to$ recurrence $\to$ closed form**.

In [ ]:
from q_kangaroo import q_petkovsek

# Step 1: Get recurrence from q-Zeilberger
zeil = q_zeilberger([(1,1,-5), (1,1,2)], [(1,1,3)], 1, 1, 4, 5, 2, 1, 3)
print("--- Step 1: q-Zeilberger finds recurrence ---")
print(f"Recurrence: {zeil['coefficients'][0]} * S(n) + {zeil['coefficients'][1]} * S(n+1) = 0")

# Step 2: Convert coefficients and solve with q-Petkovsek
coeffs = [(c.numerator, c.denominator) for c in zeil["coefficients"]]
solutions = q_petkovsek(coeffs, 2, 1)

print(f"\n--- Step 2: q-Petkovsek solves recurrence ---")
print(f"Number of solutions: {len(solutions)}")
for sol in solutions:
    print(f"Solution ratio S(n+1)/S(n): {sol['ratio']}")
    print(f"Has closed form: {sol['has_closed_form']}")
    if sol['has_closed_form']:
        print(f"  Scalar: {sol['scalar']}")
        print(f"  q-power coefficient: {sol['q_power_coeff']}")
        print(f"  Numerator factors (q^a; q)_n: {sol['numer_factors']}")
        print(f"  Denominator factors (q^b; q)_n: {sol['denom_factors']}")

--- Step 1: q-Zeilberger finds recurrence ---
Recurrence: -21/85 * S(n) + 1 * S(n+1) = 0

--- Step 2: q-Petkovsek solves recurrence ---
Number of solutions: 1
Solution ratio S(n+1)/S(n): Fraction(21, 85)
Has closed form: True
  Scalar: Fraction(1, 1)
  q-power coefficient: 0
  Numerator factors (q^a; q)_n: [('1', 6)]
  Denominator factors (q^b; q)_n: [('1', 8)]


**Interpreting the closed form.** The solution has:
- Scalar $s = 1$
- $q$-power coefficient $c = 0$ (no $q^{c\binom{n}{2}}$ factor)
- Numerator: $(q^6; q)_n$ (i.e., the factor with base $q^6$)
- Denominator: $(q^8; q)_n$ (i.e., the factor with base $q^8$)

So the closed form is $S(n) \propto (q^6; q)_n / (q^8; q)_n$. At $q = 2$, the
ratio $S(n+1)/S(n) = (1 - 2^{6+n})/(1 - 2^{8+n})$. Evaluating at $n = 0$:
$(1 - 2^6)/(1 - 2^8) = -63/-255 = 21/85$, which matches the recurrence ratio.

This confirms the q-Vandermonde identity: the sum has a product formula.

In [ ]:
# Complete pipeline in one block
print("=== Complete Pipeline: Series -> Recurrence -> Closed Form ===")
print()
print("Input: 2phi1(q^{-5}, q^2; q^3; q, q^4) at q=2")

# Step 1: Find recurrence
zeil = q_zeilberger([(1,1,-5), (1,1,2)], [(1,1,3)], 1, 1, 4, 5, 2, 1, 3)
print(f"\n1. q-Zeilberger found order-{zeil['order']} recurrence")
print(f"   Coefficients: {zeil['coefficients']}")

# Step 2: Solve recurrence
coeffs = [(c.numerator, c.denominator) for c in zeil["coefficients"]]
solutions = q_petkovsek(coeffs, 2, 1)
sol = solutions[0]
print(f"\n2. q-Petkovsek solved for closed form")
print(f"   Ratio: {sol['ratio']}")
print(f"   Numer factors: {sol['numer_factors']}")
print(f"   Denom factors: {sol['denom_factors']}")
print(f"\nResult: S(n) = (q^6; q)_n / (q^8; q)_n")

=== Complete Pipeline: Series -> Recurrence -> Closed Form ===

Input: 2phi1(q^{-5}, q^2; q^3; q, q^4) at q=2

1. q-Zeilberger found order-1 recurrence
   Coefficients: [Fraction(-21, 85), Fraction(1, 1)]

2. q-Petkovsek solved for closed form
   Ratio: Fraction(21, 85)
   Numer factors: [('1', 6)]
   Denom factors: [('1', 8)]

Result: S(n) = (q^6; q)_n / (q^8; q)_n


## Proving Nonterminating Identities

The **Chen-Hou-Mu (2010) method** proves **nonterminating** q-hypergeometric
identities by reducing them to families of terminating ones:

1. **Specialize** a parameter $a \to q^{-n}$ to create a terminating identity for each $n$.
2. **Find recurrence** via q-Zeilberger for the terminating LHS.
3. **Verify** that the RHS satisfies the same recurrence with matching initial conditions.

If both sides agree, the identity holds for all $n$ (and by analytic continuation,
for the original nonterminating parameters).

In [ ]:
from q_kangaroo import prove_nonterminating

# Prove the q-Vandermonde in nonterminating form:
# 2phi1(q^2, q^{-n}; q^3; q, q^{n+1}) = (q;q)_n / (q^3;q)_n
#
# Parameters:
#   upper_fixed = [(1,1,2)]  -- a = q^2 (n-independent)
#   n_param_offset = 0       -- b = q^{0-n} = q^{-n}
#   lower = [(1,1,3)]        -- c = q^3
#   z_pow_offset = 1         -- z = q^{1+n}
#   rhs_numer_bases = [1]    -- (q^1;q)_n in numerator
#   rhs_denom_bases = [3]    -- (q^3;q)_n in denominator
#   q = 2/1, n_test = 5, max_order = 3
result = prove_nonterminating(
    [(1, 1, 2)],  # upper_fixed
    0,             # n_param_offset
    [(1, 1, 3)],   # lower
    1,             # z_pow_offset
    [1],           # rhs numerator: (q;q)_n
    [3],           # rhs denominator: (q^3;q)_n
    2, 1,          # q = 2
    5,             # n_test = 5
    3              # max_order = 3
)
print("Proved:", result["proved"])
print("Recurrence order:", result["recurrence_order"])
print("Initial conditions checked:", result["initial_conditions_checked"])

Proved: True
Recurrence order: 1
Initial conditions checked: 2


This proves

$${}_2\phi_1\left(q^2, q^{-n}; q^3; q, q^{n+1}\right) = \frac{(q;q)_n}{(q^3;q)_n}$$

for all $n$. The algorithm found a first-order recurrence that both the LHS
(computed via q-Zeilberger) and the RHS (the Pochhammer ratio) satisfy, and
verified that 2 initial conditions match. By analytic continuation ($n \to \infty$),
this implies the nonterminating q-Vandermonde formula.

In [ ]:
# Incorrect RHS: change numerator from (q;q)_n to (q^2;q)_n
# The algorithm correctly detects the inconsistency.
result_wrong = prove_nonterminating(
    [(1, 1, 2)], 0, [(1, 1, 3)], 1,
    [2],           # WRONG: (q^2;q)_n instead of (q;q)_n
    [3],           # correct denominator
    2, 1, 5, 3
)
print("Proved:", result_wrong["proved"])
print("Reason:", result_wrong["reason"])

Proved: False
Reason: RHS does not satisfy LHS recurrence at n=3


The algorithm catches the error: the wrong RHS does not satisfy the same
recurrence as the LHS, so the identity is correctly rejected.

## Transformation Chain Discovery

`find_transformation_chain` uses **breadth-first search** over a catalog of
known transformations to connect two equivalent ${}_2\phi_1$ forms. The
transformation catalog includes:

- **Heine 1/2/3**: Three classical transformations of ${}_2\phi_1$ (DLMF 17.6)
- **Sears**: Four-term transformation for balanced terminating ${}_4\phi_3$
- **Watson**: Very-well-poised ${}_8\phi_7$ to ${}_4\phi_3$ reduction

This answers the question: *"Are these two hypergeometric series related by a
sequence of known transformations?"*

In [ ]:
from q_kangaroo import QSession, find_transformation_chain

s = QSession()

# Source: 2phi1(q^2, q^3; q^6; q, q^2)
# Target: 2phi1(q^3, q^2; q^4; q, q^3)
# These are related by Heine's first transformation.
result = find_transformation_chain(
    s,
    [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2,  # source
    [(1,1,3), (1,1,2)], [(1,1,4)], 1, 1, 3,   # target
    3, 20                                       # max_depth=3, order=20
)
print("Chain found:", result["found"])
if result["found"]:
    print("Number of steps:", len(result["steps"]))
    for i, step in enumerate(result["steps"]):
        print(f"Step {i+1}: {step['name']}")
    print("Prefactor:", result["total_prefactor"])

Chain found: True
Number of steps: 1
Step 1: heine_1
Prefactor: 1 + q^2 - q^5 - q^7 + O(q^20)


The algorithm found that Heine's first transformation connects the source and
target in a single step. The prefactor is the infinite product
$\frac{(b;q)_\infty (az;q)_\infty}{(c;q)_\infty (z;q)_\infty}$ from the
transformation formula.

In [ ]:
# No chain exists between these unrelated series
result2 = find_transformation_chain(
    s,
    [(1,1,1), (1,1,2)], [(1,1,3)], 1, 1, 1,   # 2phi1(q, q^2; q^3; q, q)
    [(1,1,5), (1,1,7)], [(1,1,10)], 1, 1, 3,   # 2phi1(q^5, q^7; q^10; q, q^3)
    2, 20
)
print("Chain found:", result2["found"])
if not result2["found"]:
    print("Max depth searched:", result2["max_depth"])

Chain found: False
Max depth searched: 2


When no chain is found within the search depth, the series may be genuinely
unrelated (not connected by classical transformations), or the required chain
may be longer than `max_depth`. Increasing `max_depth` expands the search
but grows exponentially.

## Summary: The Complete Pipeline

| Step | Tool | Purpose |
|------|------|---------|
| 1 | `q_gosper` | Check for indefinite sum (antidifference) |
| 2 | `q_zeilberger` | Find linear recurrence for definite sum |
| 3 | `verify_wz` | Independently verify the WZ proof certificate |
| 4 | `q_petkovsek` | Solve recurrence for closed-form product |
| 5 | `prove_nonterminating` | Extend to nonterminating identities |
| 6 | `find_transformation_chain` | Connect equivalent series via known transforms |

Together, these tools provide a **machine-assisted proof pipeline** for
q-hypergeometric identities. The key insight is that recurrences are the bridge:
q-Zeilberger finds them, WZ certificates verify them, and q-Petkovsek solves them.

**Next steps:**
- [hypergeometric_summation.ipynb](hypergeometric_summation.ipynb): Classical summation formulas ($q$-Gauss, $q$-Vandermonde, etc.)
- [maple_migration.ipynb](maple_migration.ipynb): Side-by-side comparison with Garvan's Maple packages